# Read Coverage and Variant Calling

---
## Before Class
1. Install bowtie2, samtools, and bamnostic: `conda install bowtie2 samtools bamnostic`
* Review bowtie2, samtools, and bamnostic documentation
* Review Counter from collections
---
## Learning Objectives

1. Use BWT algorithm to map reads to a genome
* Implement 'pileup' method
* Call variants from DNA sequencing data

Before class:
`conda install bowtie2 bamnostic samtools`

---
## Imports

In [6]:
conda install -c bioconda samtools=1.9 --force-reinstall

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /Users/alauth/opt/anaconda3/envs/b529

  added / updated specs:
    - samtools=1.9


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bzip2-1.0.8                |       h0d85af4_4         155 KB  conda-forge
    cffi-1.14.4                |   py37hbddb872_0         218 KB  conda-forge
    dbus-1.13.6                |       h2f22bb5_0         558 KB  conda-forge
    gettext-0.19.8.1           |    haf92f58_1004         3.2 MB  conda-forge
    glib-2.66.1                |       h39b9ebd_0         3.2 MB  conda-forge
    htslib-1.9                 |       h3a161e8_7         1.2 MB  bioconda
    libdeflate-1.0             |       h1de35cc_1          45 KB  bioconda
    libedit-3.1.20191231       |       hed1e85f_2         103 KB  conda-for

In [2]:
import bamnostic as bs
from collections import Counter
from scipy.stats import binom_test

---
## Mapping to a genome

In our previous class, we developed our own tool for mapping reads to a genome using the efficient BWT algorithm. Today, we will be using an existing implementation of this algorithm to align many reads to a small genome. 

For the next few classes, we will be working with a small genome that we will assume represents a sample from a diploid individual from a population. The genome itself is quite small at ~9kb and contains only a few genes to make analysis during class tractable. 


In [5]:
# We can use our original get_fasta function to examine the fasta file for the genome
from data_readers import get_fasta
file = "data/sample_genome.fna"

for name, seq in get_fasta(file):
    print(seq[1:300])

GTCTCTCTGGTTAGACCAGATCTGAGCCTGGGAGCTCTCTGGCTAACTAGGGAACCCACTGCTTAAGCCTCAATAAAGCTTGCCTTGAGTGCTTCAAGTAGTGTGTGCCCGTCTGTTGTGTGACTCTGGTAACTAGAGATCCCTCAGACCCTTTTAGTCAGTGTGGAAAATCTCTAGCAGTGGCGCCCGAACAGGGACCTGAAAGCGAAAGGGAAACCAGAGGAGCTCTCTCGACGCAGGACTCGGCTTGCTGAAGCGCGCACGGCAAGAGGCGAGGGGCGGCGACTGGTGAGTACGCC


Our first step will be creating an index using BWT so that we can align reads. To accomplish this, we could use the code from class. However, because we only allow for exact matches, we wouldn't be able to identify variants in our data. Instead, we will be using an aligner that uses the same algorithm that we implemented but allows for some mismatches to the genome called Bowtie2 ( http://bowtie-bio.sourceforge.net/bowtie2/index.shtml ). Because this is building the index of the reference genome (the Burrows-Wheeler transform), you only have to do this once for our genome.

First, create an index of the reference genome using `bowtie2-build`:

In [6]:
# This gives us the usage information for bowtie2-build
! bowtie2-build

No input sequence or sequence file specified!
Bowtie 2 version 2.3.5.1 by Ben Langmead (langmea@cs.jhu.edu, www.cs.jhu.edu/~langmea)
Usage: bowtie2-build [options]* <reference_in> <bt2_index_base>
    reference_in            comma-separated list of files with ref sequences
    bt2_index_base          write bt2 data to files with this dir/basename
*** Bowtie 2 indexes work only with v2 (not v1).  Likewise for v1 indexes. ***
Options:
    -f                      reference files are Fasta (default)
    -c                      reference sequences given on cmd line (as
                            <reference_in>)
    --large-index           force generated index to be 'large', even if ref
                            has fewer than 4 billion nucleotides
    --debug                 use the debug binary; slower, assertions enabled
    --sanitized             use sanitized binary; slower, uses ASan and/or UBSan
    --verbose               log the issued command
    -a/--noauto             disabl

In [7]:
# From above, the correct format for building an index is:
# bowtie2-build <our genome FASTA file> <name of the index we create>
! bowtie2-build data/sample_genome.fna sample_genome

Settings:
  Output files: "sample_genome.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  data/sample_genome.fna
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 2295
Using parameters --bmax 1722 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 1722 --dcv 1024
Constructing suffix-arra

Once you have created an index, you can map our reads to the genome. We have a set of simulated illumina DNA reads from this genome available in `data/sample_reads.fa`. To accomplish this, use `bowtie2` and write your files to `sample_reads.sam` using the `-S` option.

In [8]:
# This gives us the usage information for bowtie2
! bowtie2

No index, query, or output file specified!
Bowtie 2 version 2.3.5.1 by Ben Langmead (langmea@cs.jhu.edu, www.cs.jhu.edu/~langmea)
Usage: 
  bowtie2 [options]* -x <bt2-idx> {-1 <m1> -2 <m2> | -U <r> | --interleaved <i> | -b <bam>} [-S <sam>]

  <bt2-idx>  Index filename prefix (minus trailing .X.bt2).
             NOTE: Bowtie 1 and Bowtie 2 indexes are not compatible.
  <m1>       Files with #1 mates, paired with files in <m2>.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <m2>       Files with #2 mates, paired with files in <m1>.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <r>        Files with unpaired reads.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <i>        Files with interleaved paired-end FASTQ/FASTA reads
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <bam>      Files are unaligned BAM sorted by read name.
  <sam>      File for SAM output

In [9]:
# From above, the correct format for building an index is:
# bowtie2 -f -x <name of the index we created> -U <FASTA of sequence reads> -S <name of SAM output file>
# *We use -f because our input is a fasta file
! bowtie2 -f -x sample_genome -U data/sample_reads.fa -S sample_reads.sam

22952 reads; of these:
  22952 (100.00%) were unpaired; of these:
    0 (0.00%) aligned 0 times
    22519 (98.11%) aligned exactly 1 time
    433 (1.89%) aligned >1 times
100.00% overall alignment rate


You will also need to convert the sam file into a bam file using samtools sort. You can read more about samtools: http://www.htslib.org/

In [10]:
# This gives us the usage information for samtools sort
! samtools sort

Usage: samtools sort [options...] [in.bam]
Options:
  -l INT     Set compression level, from 0 (uncompressed) to 9 (best)
  -m INT     Set maximum memory per thread; suffix K/M/G recognized [768M]
  -n         Sort by read name
  -t TAG     Sort by value of TAG. Uses position as secondary index (or read name if -n is set)
  -o FILE    Write final output to FILE rather than standard output
  -T PREFIX  Write temporary files to PREFIX.nnnn.bam
      --input-fmt-option OPT[=VAL]
               Specify a single input file format option in the form
               of OPTION or OPTION=VALUE
  -O, --output-fmt FORMAT[,OPT[=VAL]]...
               Specify output format (SAM, BAM, CRAM)
      --output-fmt-option OPT[=VAL]
               Specify a single output file format option in the form
               of OPTION or OPTION=VALUE
      --reference FILE
               Reference sequence FASTA FILE [null]
  -@, --threads INT
               Number of additional threads to use [0]


In [11]:
# From above, the correct format for building the bam file is:
# samtools sort -o <name of bam file output> <name of sam file input>
! samtools sort -o sample_reads.bam sample_reads.sam

You have now mapped your reads to a reference genome using the Burrows-Wheeler algorithm! You can take a look at the sample_reads.sam file to see the plain text version of the alignments and the sample_reads.bam now contains a compressed version of the same alignments.

---
## Identifying variants in the genome

For the second section of today's class, we will be identifying variants in our diploid genome. We will be using the `bamnostic` package ( https://github.com/betteridiot/bamnostic ) to work with our aligned reads from a bam file.

To identify variants, we will test each position for non-reference alleles and perform a binomial test to determine if there is indeed a variant or just a sequencing error at that position. This algorithm is somtimes referred to as a 'pileup'.

```
find_variants(bam_file):
    For each position in genome:
        count allele frequencies (pileup)
        test for heterozygosity
        test for homozygous alternative allele
```

Note: There are multuple ways to implement this, however we recommend using `Counter()` from `collections` that has been discussed and demonstrated multiple times on the office hours live streams. The binomial test can be implemented directly from the equation below, or you can use scipy.stats.

Binomial test is calculated as: $P(X=k) = {n \choose k}p^{k}(1-p)^{n-k}$ where $k$ is the allele count, $n$ is the total number of reads, and $p$ is 0.50.

In [4]:
import bamnostic as bs
from collections import Counter
from scipy.stats import binom_test

def get_pileup(alignments, region_start = None, region_end = None):
    ''' Function build a read pileup list
        this is implemented as a list of Counter() from region_start to region_end
        with our small genome, it is reasonable to cover the entire genome
        but for larger genomes a smaller window is required.
    
    Args:
        alignments (str): bam file of alignments
        region_start (int): start position to build pileup
        region_end (int): end position to build pileup
    
    Returns:
        genome (list of Counter()): a list from region_start to region_end of
            Counters of allele frequencies
        
    Example:
        >>> get_pileup('sample_reads.bam') #doctest: +ELLIPSIS +NORMALIZE_WHITESPACE 
        [Counter({'G': 5}), Counter({'G': 8}), Counter({'T': 12}), ...] 
    '''
    # firt need to open the bam file. 
    bam = bs.AlignmentFile(alignments, "rb") 
    
    #setting the default to measuring the entire genome, from zero to the very end. 
    
    if region_start == None:
        region_start = 0
        
    if region_end == None:
        region_end = bam.header["SQ"][0]["LN"] #from the documentation and it's in the header. 
        #same concept as when we took from a header line in 575. 
    
    #list of the Counter for each base within the given region 
    genome = [Counter() for _ in range(region_start, region_end)]
    
    
    for read in bam:
        #we have an option in enumerate to say when we want to start counting from 
        #usually start 0, but we can start at a certain position, want to start at the beginning of the alignmnet of the read. 
        #increment as we go to each position. 
        for i, base in enumerate(read.seq, read.pos):
            if i >= region_start and i <= region_end:
                #increment the Counter at that base. So i is really your index. 
                genome[i-region_start].update(base) 
                #sets us back to zero being region_start. 
    
    
    # print(genome[:10])
    # print(genome[-10:])
    
    # return genome 
    

def binomial_test(major, minor):
    ''' Function to perform binomial test
        We will consider a Pvalue threshold of 0.10: 
        SNPs for which the P value of the binomial test < 0.10 failed the heterozygosity test.

    Args:
        major (int): count of most frequent allele
        minor (int): count of second most frequent allele
    
    Returns:
        is_above_threshold (bool): true if passes heterozygosity test, otherwise false
        
    Example:
        >>> binomial_test(8, 4)
        True
    '''
    #will perform binomial test 
    #Binomial test is calculated as:  𝑃(𝑋=𝑘)=(𝑛𝑘)𝑝𝑘(1−𝑝)𝑛−𝑘  where  𝑘  is the allele count,  𝑛  is the total number of reads, and  𝑝  is 0.50
    
    
    p_val = binom_test(major, major + minor, 1/2) 
    if p_val > 0.10:
        return True 
    
    #else return False. 
    return False 
    

def find_variants(reference, alignments):
    ''' Function to find variants given sequencing alignments and a reference
        Identify variants that are heterozygous using heterozygosity test
        Identify variants that are homozygous alternative allele
        
        Note: Variants are reported as 1-based coordinates
        
    Args:
        reference (str): genome reference fasta file
        alignments (str): sequence alignments
    
    Returns:
        variant_list (list of tuples): list of variants as (position, allele1, allele2)
        
    Example:
        >>> find_variants(reference = 'data/sample_genome.fna', alignments = 'sample_reads.bam') #doctest: +ELLIPSIS +NORMALIZE_WHITESPACE 
        [(240, 'A', 'G'), (354, 'G', 'A'), (803, 'C', 'A'), ...]
    '''
    
    
#     find_variants(bam_file):
#     For each position in genome:
#         count allele frequencies (pileup)
#         test for heterozygosity
#         test for homozygous alternative allele
    
    #need an empty list for our variants. 
    variant_list = []
    
    #get our genome, which we can from the reference. 
    #need genome seq to track variants. get that from the reference which is a fasta file. 
    
    
    #Reference genome right here!!
    genome = None 
    for name, seq in get_fasta(reference):
        genome = seq 
    #loops through it once to get the genome sequence. 
    
    #now need to calculate the pileup. Genome_pileup. 
    genome_pileup = get_pileup(alignments, region_start = None, region_end = len(genome))
    
    #iterate through the gneome to identify variants. 
    for j, counter in enumerate(genome_pileup, 1): #start at 1 since variants are reported as 1-based. 
        #need the reference allele 
        ref_allele = genome[j-1] #i-1 position since we're now indexing by one off. genome comes from fasta file. 
        top_alleles = counter.most_common(2) #get the top two alleles. 
        
        #test for heterozygosity need more than one at the same position. 
        if len(counter) > 1:
            if binomial_test(top_alleles[0][1], top_alleles[1][1]): #first item in the counter and second item in the counter. 
                variant_list.append((j, top_alleles[0][0], top_alleles[1][0])) #this is what they want to get out. #we want the bases themselves, not the counts.
            elif len(counter) == 0:  #there may not be any reads at a point, so we're just going to pass. 
                next 
            else: #test for homozygous, alternate site. 
                if counter[ref_allele] == 0:
                    variant_list.append((j, top_alleles[0][0], top_alleles[0][0]))
                
    return variant_list 
    
    

In [5]:
genome_02 = get_pileup('sample_reads.bam')

In [49]:
print(find_variants(reference = 'data/sample_genome.fna', alignments = 'sample_reads.bam'))

[(240, 'A', 'G'), (354, 'G', 'A'), (803, 'C', 'A'), (1411, 'A', 'G'), (1799, 'C', 'G'), (1829, 'G', 'C'), (1910, 'T', 'C'), (1958, 'A', 'T'), (2119, 'A', 'G'), (2327, 'G', 'A'), (2404, 'A', 'G'), (2425, 'T', 'A'), (2605, 'C', 'T'), (2678, 'G', 'A'), (2788, 'A', 'T'), (2965, 'A', 'C'), (3017, 'T', 'G'), (3141, 'G', 'T'), (3383, 'G', 'C'), (3536, 'G', 'A'), (3779, 'G', 'T'), (3822, 'C', 'A'), (3908, 'G', 'C'), (4497, 'C', 'T'), (4548, 'A', 'C'), (4737, 'G', 'A'), (4823, 'G', 'A'), (5144, 'C', 'G'), (5275, 'G', 'T'), (5313, 'T', 'C'), (5522, 'A', 'G'), (5586, 'G', 'A'), (5624, 'T', 'A'), (5652, 'A', 'C'), (5910, 'C', 'A'), (6013, 'A', 'T'), (6303, 'A', 'G'), (6632, 'G', 'C'), (6734, 'A', 'C'), (7099, 'C', 'T'), (7376, 'A', 'C'), (7466, 'C', 'T'), (7970, 'C', 'G'), (8116, 'G', 'T'), (8294, 'T', 'C'), (8545, 'T', 'C'), (8788, 'C', 'A'), (8981, 'G', 'C')]


In [ ]:
import doctest
doctest.testmod()

In [6]:
test1 = get_pileup("sample_reads.bam")
test1[239]

Counter({'A': 26, 'G': 24})